In [1]:
#Converting Firt Json File To CSV
import json
import csv
import pandas as pd
with open(r'C:\Users\Lenovo\Downloads\generated.json') as json_file:
    dataObj = json.load(json_file)
    
df = pd.json_normalize(dataObj)
df.columns = df.columns.map(lambda x: x.split(".")[-1])


In [2]:
#Check for unique values in dataframe
for x in range(0, 16):
    print(df.iloc[:, x].nunique())
    
for i in range(17, 22):
    print(df.iloc[:, x].nunique())
    
print(df.iloc[:,23].nunique())


2
1
994
750
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [3]:
'''Sincce there are only three variabbles which have more then one unique value.
   Therefore we take title and desc key as independent variables.
   And take isSafe as dependent variable
'''
X = df.iloc[:, 2:4].values
Y = df.iloc[:, 0].values


In [4]:
#encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_Y = LabelEncoder()
Y=labelencoder_Y.fit_transform(Y)


In [5]:
#Cleaning the teXt
import re
import nltk
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
corpus = []
for i in range(0, 1000):
    X[i][0] = re.sub('[^a-zA-z]',' ', X[i][0])
    X[i][1] = re.sub('[^a-zA-z]',' ', X[i][1])
    
    X[i][0] = X[i][0].lower()
    X[i][1] = X[i][1].lower()
    
    X[i][0] = X[i][0].split()
    X[i][1] = X[i][1].split()
    
    ps = PorterStemmer()
    X[i][0] = [ps.stem(word) for word in X[i][0] if not word in set(stopwords.words('english'))]
    X[i][1] = [ps.stem(word) for word in X[i][1] if not word in set(stopwords.words('english'))]
    
    X[i][0] =' '.join(X[i][0])
    X[i][1] =' '.join(X[i][1])


In [6]:
#Creating Bag of Words models
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X = cv.fit_transform(X.ravel()).toarray()


In [7]:
#splitting the dataset into training set & teasting set
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test= train_test_split(X[1000:2000, :],Y, test_size= 0.20, random_state=0)


In [8]:
#Part-2: Now Let's make the ANN!
#Import the keras libraries and Packages
import tensorflow
import keras
from keras.models import Sequential
from keras.layers import Dense

#Initialising the ANN
'''Sequintial package use for initialization of neural network'''
classifier = Sequential()

# Adding the Input Layer and first hidden layer
'''units means no. of nodes in hidden layers i.e. average of input nodes and output node(variables)
   eg. 11+1/2 = 6
   kernal_initializer used to give value of weights near to zero.
   activation relu means rectifier activation
   input_dim means no. of nodes for input independent variables
'''
classifier.add(Dense(units = 819, kernel_initializer   = 'uniform', activation = 'relu', input_dim = 1636))

# Adding the Second hidden layer

#Adding the output Layer
'''Here,no. of nodes use in output layer is 1.Therefore units = 1.
   If dependent variable have more than two catogories then use activation = 'softmax' 
'''
classifier.add(Dense(units = 1, kernel_initializer   = 'uniform', activation = 'sigmoid'))

#Compiling The ANN
'''If dependent variable have more than two catogories then use loss = 'categorical_crossentropy'
'''    
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

#Fitting the ANN to the Training set
classifier.fit(X_train, Y_train, batch_size = 20, epochs = 100)


Epoch 1/100
40/40 [==============================] - 1s 8ms/step - loss: 0.6844 - accuracy: 0.5711
Epoch 2/100
40/40 [==============================] - 0s 8ms/step - loss: 0.5652 - accuracy: 0.7912
Epoch 3/100
40/40 [==============================] - 0s 8ms/step - loss: 0.4101 - accuracy: 0.8916
Epoch 4/100
40/40 [==============================] - 0s 9ms/step - loss: 0.3001 - accuracy: 0.9263
Epoch 5/100
40/40 [==============================] - 0s 8ms/step - loss: 0.2310 - accuracy: 0.9483
Epoch 6/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1746 - accuracy: 0.9506
Epoch 7/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1586 - accuracy: 0.9532
Epoch 8/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1335 - accuracy: 0.9566
Epoch 9/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1225 - accuracy: 0.9478
Epoch 10/100
40/40 [==============================] - 0s 8ms/step - loss: 0.1096 - accuracy: 0.9536
Epoch 11/

In [9]:
#Part-3: Making the predictions and evaluating the model
#Predecting the test set result
y_pred =  classifier.predict(X_test)
y_pred = (y_pred > 0.5)
'''y_pred value gives which one is safe or unsafe.
'''


In [10]:
#Making an confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, y_pred)

